<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/doubt_markers_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import Cython
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/gdrive/


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [2]:

%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/1_Data Prep
%ls

/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/1_Data Prep
1_doubt_makers_word_embeddings.py
doubt_marker_list_round_3_gpt.csv
doubt_markers_dataframe_annotate.csv
doubt_markers_lexicon_dev.ipynb
doubt_markers_regex.ipynb
doubt_word_list_round_1_dw_pruned.csv
doubt_word_list_round_2.csv
doubt_words_lexicon_stem_and_similar_round1.csv
expanded_misspellings.csv
lexicon_search_and_visualizations_subreddit.Rmd
stigma_word_list_round_3_gpt.csv
trig-vectors-phrase.bin


BigQuery

In [ ]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [ ]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


     ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
784  176454       15872   139593        3     73382
889  196755       17664   123887        7     33829
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 784 to 889
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      2 non-null      Int64 
 1   SUBJECT_ID  2 non-null      Int64 
 2   HADM_ID     2 non-null      Int64 
 3   SEQ_NUM     2 non-null      Int64 
 4   ICD9_CODE   2 non-null      object
dtypes: Int64(4), object(1)
memory usage: 104.0+ bytes


Notes and patients load

In [ ]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents''')
                # LIMIT 500000

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



    ROW_ID  SUBJECT_ID  HADM_ID   CHARTDATE           CHARTTIME  \
0  1557640       22599   164788  2184-03-10 2184-03-10 04:27:00   
1  1892226       15204   168396  2128-01-25 2128-01-25 05:58:00   
2  1974608       21464   109525  2189-05-13 2189-05-13 10:56:00   
3  1993393       23286   122536  2141-07-14 2141-07-14 11:34:00   
4  1848162       11994   142976  2104-11-04 2104-11-04 06:42:00   

            STORETIME       CATEGORY DESCRIPTION   CGID  ISERROR  \
0 2184-03-10 04:38:00  Nursing/other      Report  14962     <NA>   
1 2128-01-25 06:10:00  Nursing/other      Report  15484     <NA>   
2 2189-05-13 10:58:00  Nursing/other      Report  15444     <NA>   
3 2141-07-14 11:40:00  Nursing/other      Report  14943     <NA>   
4 2104-11-04 06:51:00  Nursing/other      Report  14470     <NA>   

                                                TEXT  
0  csru nursing update\nadditional lopressor iv b...  
1  NPN\n\n\n#2Resp O- Remains in NC 600cc's 70-10...  
2  Neo Attend PE note\n

Explode by sentence, regex quote match

In [ ]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)



<ipython-input-89-1e5fd6deed48>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])


# Regex matching all stemwords + any semantically similar word identified

In [3]:
# Read in sentence tokenized MIMIC-III

full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")



<ipython-input-3-70d50a644935>:3: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataframe = pd.read_csv("/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/mimic_iii_sentence_tokenized.csv")


In [4]:


doubt_word_list = pd.read_csv("doubt_marker_list_round_3_gpt.csv")


# Create regex which is each stem word + all similar_words identified in the word embeddings step, which will be used to search the charts
def group_and_create_regex(dataframe):
    grouped_data = dataframe.groupby('stem_word')['similar_word'].apply(list).reset_index()
    grouped_data['regex'] = grouped_data.apply(lambda row: '|'.join([re.escape(row['stem_word'])] + [re.escape(word) for word in row['similar_word']]), axis=1)
    return grouped_data

grouped_data = group_and_create_regex(doubt_word_list)
print(grouped_data)

# Create function that takes regex from each row and searches the charts

def identify_matching_strings(full_dataframe, grouped_data):
    matching_dataframes = []

    for index, row in grouped_data.iterrows():
        regex = row['regex']
        terms = row['similar_word']
        for term in terms:
            term = re.escape(term)
            matches = full_dataframe.loc[full_dataframe['Sentence'].str.contains(term, flags=re.IGNORECASE, na=False)].copy()
            matches['regex'] = regex
            matches['matched_term'] = term  # Add the matched term as a new column
            matching_dataframes.append(matches)

    if matching_dataframes:
        return pd.concat(matching_dataframes, axis=0)
    else:
        return pd.DataFrame()



result = identify_matching_strings(full_dataframe, grouped_data)
result



     stem_word                                       similar_word  \
0      adamant         [adamantly, adamant/belligerant, addamant]   
1    allegedly  [alleged, supposedly, undisguisedly, purported...   
2     chat_gpt  [ "skeptical,  dubiousness,  questionable,  do...   
3      claimed  [claiming, proclaimed, claimes, claimedthat, a...   
4  disbelieves  [disbelieve, misbelieves, disbelieved, disbeli...   
5      dubious  [doubious, 'dubious, non-dubious, dubiously, d...   
6      insists  [insisting, reinsists, insist, insisted, .insi...   

                                               regex  
0     adamant|adamantly|adamant/belligerant|addamant  
1  allegedly|alleged|supposedly|undisguisedly|pur...  
2  chat_gpt|\ "skeptical|\ dubiousness|\ question...  
3  claimed|claiming|proclaimed|claimes|claimedtha...  
4  disbelieves|disbelieve|misbelieves|disbelieved...  
5  dubious|doubious|'dubious|non\-dubious|dubious...  
6  insists|insisting|reinsists|insist|insisted|\....  


Sentence ID  ROW_ID_x  SUBJECT_ID   HADM_ID   CHARTDATE  \
1276939       1276939     19031       23028  124346.0  2140-12-22   
1400541       1400541     25450       91855  174847.0  2113-01-30   
1420518       1420518    698977       85435  166741.0  2118-08-25   
1420572       1420572    698968       85435  166741.0  2118-08-25   
1420605       1420605    699024       85435  166741.0  2118-08-25   
...               ...       ...         ...       ...         ...   
18269695     18269695   1353463        6886  145695.0  2162-07-30   
18288152     18288152   1355412        6976  186850.0  2146-10-17   
15590159     15590159   1639737       29115  160056.0  2119-10-08   
10624754     10624754   1400083       10422  113048.0  2123-07-19   
10624754     10624754   1400083       10422  113048.0  2123-07-19   

                    CHARTTIME            STORETIME           CATEGORY  \
1276939                   NaN                  NaN  Discharge summary   
1400541                   NaN                  NaN  Discharge summary   
1420518   2118-08-25 03:49:00  2118-08-25 06:36:58            Nursing   
1420572   2118-08-25 03:49:00  2118-08-25 05:46:03            Nursing   
1420605   2118-08-25 15:29:00  2118-08-25 15:29:20            Nursing   
...                       ...                  ...                ...   
18269695  2162-07-30 15:45:00  2162-07-30 16:22:00      Nursing/other   
18288152  2146-10-17 13:11:00  2146-10-17 13:42:00      Nursing/other   
15590159  2119-10-08 18:21:00  2119-10-08 18:38:00      Nursing/other   
10624754  2123-07-19 04:36:00  2123-07-19 05:02:00      Nursing/other   
10624754  2123-07-19 04:36:00  2123-07-19 05:02:00      Nursing/other   

                    DESCRIPTION     CGID  ...  ROW_ID_y GENDER         DOB  \
1276939                  Report      NaN  ...     21738      F  2101-11-13   
1400541                  Report      NaN  ...     44411      M  2049-03-02   
1420518   Nursing Progress Note  17114.0  ...     42724      F  2030-10-18   
1420572   Nursing Progress Note  17114.0  ...     42724      F  2030-10-18   
1420605   Nursing Transfer Note  17600.0  ...     42724      F  2030-10-18   
...                         ...      ...  ...       ...    ...         ...   
18269695                 Report  15451.0  ...      6512      M  2111-04-13   
18288152                 Report  17574.0  ...      6599      M  1844-12-29   
15590159                 Report  21291.0  ...     27419      F  2063-11-21   
10624754                 Report  15666.0  ...      9862      M  2056-09-13   
10624754                 Report  15666.0  ...      9862      M  2056-09-13   

                 DOD    DOD_HOSP     DOD_SSN EXPIRE_FLAG  \
1276939          NaN         NaN         NaN           0   
1400541          NaN         NaN         NaN           0   
1420518          NaN         NaN         NaN           0   
1420572          NaN         NaN         NaN           0   
1420605          NaN         NaN         NaN           0   
...              ...         ...         ...         ...   
18269695         NaN         NaN         NaN           0   
18288152  2146-12-14  2146-12-14  2146-12-14           1   
15590159         NaN         NaN         NaN           0   
10624754  2123-09-21  2123-09-21  2123-09-21           1   
10624754  2123-09-21  2123-09-21  2123-09-21           1   

                                                   Sentence  \
1276939    The patient adamantly requested discharge, re...   
1400541   The only other options would be to reevaluate ...   
1420518           Pt adamantly refusing foley at this time.   
1420572           Pt adamantly refusing foley at this time.   
1420605           Pt adamantly refusing foley at this time.   
...                                                     ...   
18269695  NURSING NOTE SEE CAREVUE FOR SPECIFICS: NEURO-...   
18288152   1:1 SITTER AT NITE, SECONDARY TO SUNDOWNING [...   
15590159   [**Name (NI) 4**] son and husband [**Name (NI...   
10624754  NURSIN

In [5]:

quoted_dataframe = result

quoted_dataframe["doubt_testimony"] = ""
quoted_dataframe["annotator_comments"] = ""
# 1459 matching sentences from total of 9,118,501 sentences from 366,002 charts
# 4,418 unique sentences from total of 30,582,959 sentences from 1,274,317 charts
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("doubt_markers_dataframe_annotate.csv")



Explore the Data


In [ ]:
#from google.colab import runtime
#runtime.unassign()